# Capítulo 8 - Modelando Dados

## Trabalhando com Páginas HTML

### Analisando XML e HTML

In [2]:
from lxml import objectify
import pandas as pd
from distutils import util

xml = objectify.parse(open('XMLData.xml'))
root = xml.getroot()
df = pd.DataFrame(columns = ('Number', 'Boolean'))

for i in range(0,4):
    obj = root.getchildren()[i].getchildren()
    row = dict(zip(['Number', 'Boolean'],
                  [obj[0].pyval,
                  bool(util.strtobool(obj[2].text))]))
    row_s = pd.Series(row)
    row_s.name = obj[1].text
    df = df.append(row_s)
    
print(type(df.loc['First']['Number']))
print(type(df.loc['First']['Boolean']))


<class 'int'>
<class 'bool'>


C:\Users\55279\AppData\Local\Temp\ipykernel_4572\1208396869.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_s)
C:\Users\55279\AppData\Local\Temp\ipykernel_4572\1208396869.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_s)
C:\Users\55279\AppData\Local\Temp\ipykernel_4572\1208396869.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_s)
C:\Users\55279\AppData\Local\Temp\ipykernel_4572\1208396869.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_s)


Este código tem como objetivo ler um arquivo XML, extrair informações específicas e armazená-las em um dataframe do Pandas. Ele utiliza a biblioteca lxml para analisar o arquivo XML e a biblioteca Pandas para armazenar as informações em um formato de tabela. O código extrai informações numéricas e booleanas de cada nó do arquivo XML e as armazena em um dataframe com duas colunas, "Number" e "Boolean". O objetivo final é permitir que as informações do arquivo XML sejam manipuladas e analisadas com facilidade usando a biblioteca Pandas.

### Usando o XPath para extração de dados

In [2]:
from lxml import objectify
import pandas as pd
from distutils import util

xml = objectify.parse(open('XMLData.xml'))
root = xml.getroot()

map_number = map(int, root.xpath('Record/Number'))
map_bool = map(str, root.xpath('Record/Boolean'))
map_bool = map(util.strtobool, map_bool)
map_bool = map(bool, map_bool)
map_string = map(str, root.xpath('Record/String'))

data = list(zip(map_number, map_bool))

df = pd.DataFrame(data,
                 columns=('Number', 'Boolean'),
                 index = list(map_string))

print(df)
print(type(df.loc['First']['Number']))
print(type(df.loc['First']['Boolean']))


        Number  Boolean
First        1     True
Second       2    False
Third        3     True
Fourth       4    False
<class 'numpy.int64'>
<class 'numpy.bool_'>


## Trabalhando com Texto Puro

### Lidando com Unicode

In [3]:
# Lembre de sempre usar UTF-8 ao trabalhar com Python

### Stemizando e removendo palavras vazias

Stemização é o processo de reduzir uma palavra ao seu radical, removendo sufixos e prefixos, a fim de obter sua forma básica. O objetivo é obter uma forma canônica para palavras relacionadas, facilitando a comparação e a busca de informações em textos.

Ex: gato, gata, gatinho serão reduzidos para o radical "gat"

É uma técnica fundamental em Data Science para pré-processamento de dados de texto em NLP, que ajuda a tornar o processo de análise de dados mais preciso, eficiente e escalável.

## Usando o Modelo Saco de Palavras e Mais

Técnica de processamento de linguagem natural que consiste em representar um texto como um conjunto de palavras, sem levar em consideração a ordem em que elas aparecem. Cada palavra é tratada como uma "feature" ou variável e a frequência de cada palavra no texto é utilizada para criar um vetor de características que representa o texto. Esse modelo é usado em tarefas como classificação de textos e mineração de sentimentos, entre outras aplicações.

Antes de aplicar o modelo saco de palavras, é comum realizar algumas práticas de pré-processamento de texto, como:

1- Remover caracteres indesejados, como pontuação, números, caracteres especiais, etc.

2- Converter todo o texto para minúsculas (ou maiúsculas) para que palavras iguais, mas com letras maiúsculas ou minúsculas diferentes, sejam tratadas como iguais.

3- Remover palavras irrelevantes (stopwords), que são palavras comuns que não adicionam nenhum valor semântico ao texto, como "a", "o", "de", "para", etc.

4- Realizar a lematização ou a stemização das palavras, que é a redução das palavras para a sua forma básica (radical), de forma a agrupar palavras semelhantes.

5- Realizar a tokenização do texto, que é a divisão do texto em tokens (palavras) individuais para que possam ser contados e representados pelo modelo saco de palavras.

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import *

categories = ['comp.graphics', 'misc.forsale',
             'rec.autos', 'sci.space']
twenty_train = fetch_20newsgroups(subset = 'train',  #coleção de aproximadamente 20.000 documentos de grupos de discussão Usenet
                                 categories=categories,
                                 shuffle=True,
                                 random_state=42)

count_vect = CountVectorizer()                 # transforma os documentos de texto em uma matriz de frequência de palavras.
X_train_counts = count_vect.fit_transform(
    twenty_train.data)

print("BOW shape:", X_train_counts.shape)
caltech_idx = count_vect.vocabulary_['caltech']
print('"Caltech": %i' % X_train_counts[0, caltech_idx])

BOW shape: (2356, 34750)
"Caltech": 3


Em resumo, o código carrega os dados de treinamento do conjunto de dados 20 Newsgroups, constrói uma matriz de frequência de palavras usando o CountVectorizer e exibe a forma da matriz resultante, bem como a contagem de ocorrências da palavra "caltech" no primeiro documento.

## Trabalhando com n-gramas

Em processamento de linguagem natural, n-gramas são sequências contíguas de n elementos (geralmente palavras) em um texto. Os n-gramas são usados para análise estatística e modelagem de linguagem, permitindo extrair informações sobre a estrutura e o contexto dos textos. Por exemplo, um 2-grama (ou bigrama) em um texto como "O gato preto" seria: "O gato", "gato preto".

In [3]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

categories = ['sci.space']

twenty_train = fetch_20newsgroups(subset='train',
                                 categories=categories,
                                 remove=('headers',
                                         'footers',
                                         'quotes'),   # remover cabeçalhos, rodapés e citações dos documentos
                                 shuffle=True,
                                 random_state=42)

count_chars = CountVectorizer(analyzer='char_wb',
                              ngram_range=(3,3),
                              max_features=10)

X = count_chars.fit_transform(twenty_train.data)

feature_names = count_chars.get_feature_names_out()

print(feature_names)
print(X[1].todense())


[' an' ' in' ' of' ' th' ' to' 'he ' 'ing' 'ion' 'nd ' 'the']
[[0 0 2 5 1 4 2 2 0 5]]


Em resumo, o código analisa os textos do conjunto de dados, extrai os n-gramas de caracteres e palavras mais frequentes e exibe essas informações.